# **Modélisation des distributions d’espèces** <br>*Les effets de priorité et la limitation de la dispersion affectent-ils les possibilités d’inférence des déterminants de la niche ?*
*Présentation du 21 mars 2023*

## **Introduction**

<p style='text-align: justify;'>Dans le contexte actuel de changements globaux (changements climatiques, augmentation de la connectivité entre les territoires, ...), la compréhension des facteurs qui régissent la distribution des espèces est devenue une problématique majeure en écologie afin de protéger la biodiversité. À quel point l'aire de répartition d'une espèce de lemming va diminuer en raison du réchauffement de l'Arctique ? Quels sites risquent d'être colonisés par une plante invasive ? De fait, déterminer les paramètres de la <b>niche écologique</b> d'une espèce est une idée attirante. Plusieurs méthodes de <b>modélisation de la distribution des espèces</b> ont été développées au cours des dernières décennies pour tenter d'inférer ces paramètres. Le but de mon stage est d'étudier l'impact de deux concepts issus de l'écologie des communautés, les <b>effets de priorité</b> et la <b>limitation de la dispersion</b>, sur les performances des <b>modèles de distribution conjointe des espèces</b>.</p>

### 1. *La niche écologique*

<p style='text-align: justify;'>Imaginons une espèce quelconque, et un paysage qui comporte plusieurs sites avec des caractéristiques différentes. La capacité de cette espèce à survivre dans un site en particulier est déterminée par sa réponse aux caractéristiques de ce site. Par exemple, est-ce que l'exposition au soleil du site A est suffisante pour permettre à une certaine espèce de plante de se développer ? Est-ce que la température en hiver du site B est assez élevée pour permettre à une certaine espèce de reptile de survivre ? L'ensemble des valeurs des caractéristiques environnementales qui permettent à l'espèce de subsister dans un milieu est la <b>niche fondamentale</b> de cette espèce (<b>Box 1</b>).<br>Le concept de niche fondamentale ne prend en compte que les <b>facteurs abiotiques</b>. La présence d'une espèce dans un site ne peut cependant pas se résumer à ces facteurs, et il est nécessaire de considérer les <b>facteurs biotiques</b>, c'est-à-dire l'impact des autres espèces présentes sur le site. Est-ce qu'un prédateur A, plus grand et plus fort q'un prédateur B, va empêcher la présence de ce dernier dans un site par exclusion compétitive ? Ou est-ce qu'au contraire la présence d'une certaine espèce de pollinisateur dans un site va permettre à une certaine espèce de plante de se développer en facilitant sa reproduction ? La présence effective d'une espèce dans un site pourtant considéré comme adéquat selon sa niche fondamentale est ainsi conditionnée par les <b>interactions interspécifiques</b>. Cette limitation des sites selon les facteurs biotiques s'appelle la <b>niche réalisée</b> de l'espèce. En limitant encore plus cette niche réalisée par la capacité de dispersion de l'espèce, c'est-à-dire sa possibilité d'accéder à de nouveaux sites dans lesquels elle est capable de survivre, nous obtenons la <b>distribution de l'espèce</b> (<b>Figure 2</b>). </p>

<div class="alert alert-block alert-info">
<b>Box 1 : Niche fondamentale</b> <br>
<p style='text-align: justify;'>La <b>niche fondamentale</b> d’une espèce peut se définir comme un <b>hypervolume à n dimensions</b>, où chaque dimension représente une variable environnementale qui a un impact sur la survie de l’espèce (Hutchinson, 1957 ; <b>Figure 1</b>). Les effets de ces variables sont ce qu'on appelle les <b>déterminants de la niche</b>.</p>

<figure>
<center><img src="Niche.png" style="width:70%" /></center>
<figcaption align = "center"> <b>Figure 1.</b> Représentation de la niche fondamentale d'une espèce définie par <b>A.</b> une variable environnementale (une dimension) et <b>B.</b> deux variables environnementales (deux dimensions). Dans l'exemple en une dimension, la survie de l'espèce est maximale lorsque la valeur de la variable environnementale est 10. Dans l'exemple en deux dimensions, la survie de l'espèce est maximale lorsque les valeurs des deux variables environnementales sont 10.</figcaption>
</figure>
</div>



<figure>
<center><img src="Hutchinson.png" style="width:30%" /></center>
<figcaption align = "center"> <b>Figure 2.</b> Diagramme BAM (Biotique Abiotique Mouvement). <b>A</b> correspond à l’ensemble des sites où les conditions abiotiques sont favorables à la présence de l’espèce et correspond à la niche fondamentale selon Hutchinson (<b>NF</b>). <b>B</b> correspond à l’ensemble des sites où les conditions biotiques sont favorables à la présence de l’espèce. <b>M</b> correspond à la capacité de l’espèce de disperser vers un site à partir d’une zone occupée. L’intersection entre <b>A</b> et <b>B</b> correspond à la niche réalisée selon Hutchinson (<b>NR</b>). L’intersection <b>P</b> entre <b>A</b>, <b>B</b> et <b>M</b> est la distribution réelle de l’espèce. </figcaption>
</figure>

### 2. *La modélisation de la distribution des espèces*

<p style='text-align: justify;'>Les <b>Modèles de Distribution d'Espèces (Species Distribution Models, SDM)</b>, développés depuis la fin du XXe siècle, reposent sur le concept de la niche fondamentale vu précédemment. Ils ont pour but de mettre en relation des données sur la distribution d'une espèce obtenues sur le terrain (abondance, présence-absence ou présence seule) avec les variables environnementales des  sites étudiés afin de déterminer quelles conditions abiotiques sont favorables à la présence de l'espèce. En déterminant par la suite quelles zones géographiques correspondent à ces conditions, il est possible d'obtenir la distribution potentielle de l'espèce (<b>Box 2</b>). Or, ces SDM omettent les éventuelles interactions interspécifiques, qui peuvent jouer sur la présence ou non d'une espèce sur un site. Les <b>déterminants de la niche</b> estimés par ces modèles sont ainsi biaisés, puisque les effets des facteurs biotiques sont mélangés avec les effets des facteurs abiotiques. Une première méthode pour corriger cela a été d'inclure la présence d'autres espèces comme prédicteurs lors du calcul de SDM. Cette option implique cependant une bonne connaissance de la distribution des espèces utilisées comme prédicteurs, et une relation unidirectionnelle dans laquelle la distribution de l'espèce étudiée à l'aide du SDM n'a pas d'effet sur la distribution de ces espèces, ce qui n'est pas réaliste.<br></p>

<div class="alert alert-block alert-info">
<b>Box 2 : Exemple d'application des SDM : Le lemming à collier face au changement climatique</b> <br>
<head>
  <title>Cols</title>
  <style>
    #left {
      width: 300px;
      float: left;
    }
    #right {
      margin-left: 290px;
    }
    .clear {
      clear: both;
    }
  </style>
</head>

<body>
  <div id="container">
    <div id="left">
    <p style='text-align: justify;'> Au cours de mon stage de M1 effectué dans l'UMR BIOGEOSCIENCES de l'Université de Bourgogne-Franche-Comté, j'ai travaillé sur la modélisation de la distribution de plusieurs espèces de rongeurs des milieux arctiques et boréaux, et sur la prédiction des changements de leur aire de répartition au cours du XXIe siècle (<b>Figure 3</b>). Le principe est d'entrainer 2 SDM basés sur le Machine Learning (BRT et MaxEnt) à l'aide de données climatiques collectées entre 1970 et 2000, pour obtenir une estimation de l'effet des variables sur la distribution des espèces. Ces modèles sont ensuite appliqués à des données environnementales simulées à partir des scénarios du GIEC (base de données WORLDCLIM) pour prédire les distributions futures de ces espèces.
    (<b>Figure 3</b>).
    </p>
    </div>
    <div id="right">
    <figure>
    <center><img src="SDM.png" style="width:100%" /></center>
    <figcaption align = "center"> <b>Figure 3.</b> Prédictions de l'aire de distribution du lemming à collier <i>Dicrostonyx torquatus</i> par les algorithmes BRT et MaxEnt. </figcaption>
    </figure>
    </div>
    <div class="clear"></div>
  </div>
</body>
</div>

<p style='text-align: justify;'>Les <b>Modèles de Distribution Conjointe des Espèces (Joint Species Distribution Models, JSDM)</b> ont pour principe de modéliser les effets de variables environnementales sur les données de distributions de plusieurs espèces en même temps, contrairement aux SDM qui les modélisent une espèce à la fois. Ceci permet d'obtenir, en plus de l'estimation de ces effets, une matrice de covariances résiduelles entre les espèces, qui correspondent à des co-occurences. Une co-occurence positive entre deux espèces signifient qu'elles sont plus souvent présentes ensemble dans un site que ce que les variables environnementales seules prédisent. À l'inverse, une co-occurence négative indique qu'elles sont moins souvent présentes ensemble qu'attendu. Ces co-occurences peuvent être issus d'erreurs dans l'estimation des paramètres du modèle, de l'absence d'une variable environnementale d'intérêt ou encore d'interactions entre les deux espèces (<b>Box 3</b>). Il est impossible d'inférer avec certitude ces interactions, mais cela représente une avancée par rapport aux SDM.
</p>

<div class="alert alert-block alert-info">
<b>Box 3 : Exemple d'interprétation des JSDM : Les mouches des fruits de La Réunion (Facon <i>et al.</i>, 2021)</b> <br>
<head>
  <title>Cols</title>
  <style>
    #left {
      width: 300px;
      float: left;
    }
    #right {
      margin-left: 290px;
    }
    .clear {
      clear: both;
    }
  </style>
</head>

<body>
  <div id="container">
    <div id="left">
    <p style='text-align: justify;'>L'article de Facon <i>et al.</i> (2021) s'intéresse à la distribution de 8 espèces de mouches des fruits à La Réunion (4 généralistes, 3 spécialistes des Cucurbitacées et 1 spécialiste des Solanacées), et montre un effet conjoint de l'environnement, de la plante hôte (<b>Figure 4</b>) et de la compétition (<b>Figure 5</b>) sur la distribution de ces espèces. Pour l'effet de la plante hôte, la fitness mesurée en laboratoire peut s'interpréter comme la niche fondamentale, et la réponse obtenue avec le modèle comme une inférence de la niche réalisée (<b>Figure 4</b>). Les différences entre les matrices de covariances résiduelles du modèle nul et du modèle prenant en compte les variables environnementales et la plante hôte (<b>Figure 5</b>) montrent l'importance d'inclure l'ensemble des variables d'intérêt dans le modèle. En ajoutant les variables, la matrice peut s'interpréter comme indiquant qu'il y a compétition au sein des généralistes et au sein des spécialistes. Les généralistes n'étant pas présents sur les mêmes fruits que les spécialistes dans les données récoltées sur le terrain, le modèle ne peut pas détecter l'exclusion compétitive déduite de la figure 4.
    </p>
    </div>
    <div id="right">
    <figure>
    <center><img src="FruitFly1.png" style="width:80%" /></center>
    <figcaption align = "center"> <b>Figure 4.</b> Utilisation des plantes par les différentes espèces de mouches des fruits étudiées et réponse estimée par le JSDM. </figcaption>
    </figure>
    <figure>
    <center><img src="FruitFly2.png" style="width:100%" /></center>
    <figcaption align = "center"> <b>Figure 5.</b> Matrices de covariances résiduelles entre les espèces pour (a) le modèle nul et (b) le modèle prenant en compte les variables environnementales et l'espèce de plante hôte.</figcaption>
    </figure>
    </div>
    <div class="clear"></div>
  </div>
</body>
</div>

### 3. *Tester les effets issus de processus d'écologie des communautés*


<p style='text-align: justify;'>Les JSDM semblent prometteurs pour améliorer notre capacité à prédire la distribution des espèces. La <b>dynamique de la disponibilité des sites</b> peut cependant avoir un effet sur les données collectées sur le terrain, qui servent à ajuster ces modèles. Dans un paysage, les sites ne sont pas tous toujours disponibles, et la mise à disposion d'un site permet la mise en place de processus qui ajoutent de l'aléatoire dans la présence des espèces. Les premiers processus qui nous intéressent sont les <b>effets de priorité</b>. Lorsqu'un site devient disponible, par exemple un arbre qui fructifie ou une mare qui se remplit à la saison des pluie, la première espèce qui colonise ce site peut avoir un avantage sur les autres. Ceci est évident pour les espèces sessiles, qui vont occuper physiquement l'espace, mais d'autres processus peuvent être impliqués. Une espèce de mouche des fruits qui arrive en première sur un arbre qui vient de fructifier peut par exemple déposer des phéromones sur les fruits qu'elle compte utiliser pour pondre, excluant ainsi les espèces qui arriveraient après. Des espèces qui sont de moins bons compétiteurs peuvent ainsi subsister dans la méta-communauté à l'aide de ces effets priorité. L'autre processus d'intérêt est la <b>limitation de la dispersion</b>. Cela consiste à considérer que la dispersion des différentes espèces au sein la méta-communauté est suffisamment faible pour que toutes les espèces n'arrivent pas en même temps lorsqu'un site devient disponible, permettant ainsi aux effets de priorité de survenir. Cela permet de créer de l'aléatoire dans la composition en espèce des sites et, de fait, les données collectées sur un site peuvent varier selon l'ordre d'arrivée des espèces sur ce site.</p>


<p style='text-align: justify;'>L'idée derrière ce stage est de tester à quel point ces processus ont un effet notable sur les performances des JSDM, c'est-à-dire leur capacité à inférer correctement les paramètres de la niche fondamentale d'une espèce. Une première étape est de simuler une méta-communauté où l'ensemble des paramètres des niches fondamentales des espèces sont connus et dans laquelle ces processus sont présents. Une seconde étape est d'étudier les résultats de JSDM calculés à partir de jeux de données issus de cette méta-communauté théorique.</p>

## **I. Simulation d'une méta-communauté**

Afin de tester l'impact des effets de priorité sur les JSDM, nous avons besoin de simuler des données d'une <b>méta-communauté</b> dans laquelle les différents paramètres sont connus. Soit une méta-communauté avec <b>E espèces</b>, répartie sur <b>X sites</b>. Nous définissons notre modèle en temps discret, où une génération équivaut à un pas de temps, tel que si $N_{ix}(t)$ est le nombre d'individus de l'espèce $i$ sur le site $x$ au pas de temps $t$, alors le nombre d'individus de l'espèce $i$ sur le site $x$ au pas de temps $t+1$ s'écrit :
$$N_{ix}(t+1)=(N_{ix}(t)+Nb_{ix}(t))(1-\mu_i)-D_{ix}(t)+I_{ix}(t)$$
avec $Nb_{ix}(t)$ le nombre de nouveaux individus produits pendant le pas de temps $t$, $\mu_i$ le taux de mortalité de l'espèce $i$, $D_{ix}(t)$ le nombre d'individus de l'espèce $i$ qui dispersent depuis le site $x$ au pas de temps $t$, et $I_{ix}(t)$ le nombre de migrants de l'espèce $i$ qui arrivent sur le site $x$ au pas de temps $t$.

### 1. *Caractérisation de la méta-communauté*

<p style='text-align: justify;'>Pour conserver un modèle simple, nous considérons un paysage avec <b>100 sites équidistants</b> les uns des autres. Ces sites ne sont pas disponibles tout le temps, et suivent un régime d'extinction-allumage aléatoire qui fait qu'en moyenne seulement la moitié de ces sites est disponible à chaque pas de temps. Cela veut dire qu'à chaque pas temps, chaque site allumé a une certaine probabilité p de s'éteindre, et chaque site éteint a une probabilité p de s'allumer. Les sites ne diffèrent entre eux que par la ressource qu'ils comportent. Nous avons <b>2 ressources</b> différentes, présentes chacune dans 50 sites. La niche fondamentale des espèces présente dans le paysage correspond donc à leur capacité à exploiter chacune des ressources. Pour la suite, nous considérons <b>2 espèces</b> qui se partagent ces 2 ressources. L'<b><font color='blue'>espèce 1</font></b>, généraliste, peut utiliser la <b><font color='green'>ressource 1</font></b> et la <b><font color='red'>ressource 2</font></b> de façon équivalente. L'<b><font color='orange'>espèce 2</font></b> est spécialiste de la <b><font color='red'>ressource 2</font></b>. Le <b>compromis</b> dans l'efficacité de l'exploitation de chacune des 2 ressources par ces espèces est défini par l'équation :</p>

$$(e_{i1})^{1/s}+(e_{i2})^{1/s}=1$$
avec $e_{i1}$ l'efficacité de l'exploitation de la ressource 1 par l'espèce $i$, $e_{i2}$ l'efficacité de l'exploitation de la ressource 2 par l'espèce $i$, et $s$ la force du compromis (<b>Box 4</b>).

<div class="alert alert-block alert-info">
<b>Box 4 : Compromis dans l'exploitation des ressources</b> <br>
<p style='text-align: justify;'>
Imaginons un <b>trait</b> lié à l'exploitation d'une ressource (par exemple la longueur du bec d'un oiseau ou le microbiote instestinal d'un ruminant). L'exploitation optimale de cette ressource correspond à une certaine valeur de ce trait (<b>Figure 6 A.</b>). Si une espèce est confrontée à plusieurs ressources nécessitant des valeurs de trait différentes pour être exploitées de façon optimale, elle ne pourra se spécialiser que sur l'exploitation d'une seule de ces ressources. Si elle souhaite exploiter plusieurs ressources, c'est-à-dire être un généraliste, elle ne pourra pas le faire de façon optimale (<b>Figure 6 B.</b>). Ceci constitue un <b>compromis dans l'exploitation des ressources</b>.
</p>
<figure>
<center><img src="Ressource.png" style="width:80%" /></center>
<figcaption align = "center"> <b>Figure 6.</b> Paysage adaptatif pour l'exploitation de deux ressources. <b>A.</b> Soit un trait pouvant varier entre 0 et 5, et 2 ressources. La <b>valeur optimale</b> du trait pour exploiter la ressource 1 est 1.75. La valeur optimale du trait pour exploiter la ressource 2 est 3.25. <b>B.</b> Soit une espèce généraliste qui a une valeur de trait de 2.5, et un spécialiste de la ressource 2 qui a une valeur de trait de 3.25. Ici, le généraliste a une efficacité de 0.35 pour l'exploitation des 2 ressources, tandis que le spécialiste a une efficacité de 1 pour l'exploitation de la ressource 2 et une efficacité presque nulle pour l'exploitation de la ressource 1.
</figcaption>
</figure>
</div>

<p style='text-align: justify;'>Dans notre système à 2 ressources, afin de simplifier le modèle, nous considérons qu'un spécialiste d'une ressource ne peut absolument pas exploiter l'autre, et qu'un généraliste peut exploiter les 2 ressources de façon équivalente. l'équation présentée précedemment correspond à cela (<b>Figure 7</b>). Cette équation nous permet de calculer un <b>coefficient d'exploitation g</b> pour notre espèce généraliste, que nous exploitons plus tard pour déterminer dans quels cas les effets de priorité sont possibles.
</p>
<figure>
<center><img src="Tradeoff.png" style="width:80%" /></center>
<figcaption align = "center"> <b>Figure 7.</b> Courbes de compromis pour plusieurs forces de compromis s. <b>A.</b> Lorsque <b>s < 1</b>, le compromis entre l'efficacité pour exploiter la ressource 1 et l'efficaité pour exploiter la ressource 2 est dit <b>faible</b>. Lorsque <b>s > 1</b>, ce compromis est au contraire dit <b>fort</b>. <b>B.</b> Position d'un spécialiste absolu de la ressource 2 (exploitation optimale de la ressource 2 et nulle de la ressource 1) et d'un généraliste absolu (efficacité de l'exploitation <b>g</b> identique dans les 2 ressources) sur les courbes de compromis. Le spécialiste est toujours en (0,1), tandis que le généraliste est à l'intersection entre la courbe de compromis et la droite y = x.</figcaption>
</figure>

<p style='text-align: justify;'><b>Bilan : </b>Notre méta-communauté se compose de 2 espèces, un généraliste capable d'exploiter 2 ressources et un spécialiste d'une de ces ressources. Ces ressources sont présentes chacune sur 50 sites, pour un total de 100 sites. Ces sites sont équidistant, donc un individu qui disperse d'un site peut arriver dans n'importe quel site avec la même probabilité. Chaque site peut être éteint ou allumé de façon aléatoire.
</p>

### 2. *Dynamique intra-site*

La dynamique au sein de chaque site à chaque génération se divise en 3 phases : la reproduction, la survie et la dispersion.<br>
Pour déterminer le nombre d'individus produits par la reproduction, nous utilisons un modèle de compétition de Lotka-Volterra (<b>Box 5</b>). L'équation de ce modèle est :
$$Nb_{ix}(t) = N_{ix}(t)r_i\biggl(1 - \frac{\sum_{j=1}^E\alpha_{ij}N_{jx}(t)}{K_{ix}}\biggr)$$
avec $Nb_{ix}(t)$ le nombre de nouveaux individus de l'espèce $i$ sur le site $x$ au pas de temps $t$, $N_{ix}(t)$ le nombre d'individus de l'espèce $i$ sur le site $x$ au début du pas de temps $t$, $r_i$ le taux de croissance maximale de l'espèce $i$, $\alpha_{ij}$ l'effet de l'espèce $j$ sur l'espèce $i$ (le coefficient d'interaction), $N_{jx}(t)$ le nombre d'individus de l'espèce $j$ sur le site $x$ au début du pas de temps $t$ et $K_{ix}$ le nombre maximum d'individus de l'espèce $i$ que le site $x$ est capable d'accueillir (la capacitance du milieu). Ce $K_{ix}$ est défini est multipliant une capacité basale $K_0$ (choisie pour l'ensemble des ressources) par le coefficient d'exploitation de la ressource de l'espèce. Pour notre spécialiste, nous avons $K = K_0$ dans les sites comportant la ressource 2, et $K = 0$ dans les sites comportant la ressource 1. Pour notre généraliste, nous avons $K = g*K_0$ dans l'ensemble des sites. Pour simplifier le modèle, nous considérons que les coefficients d'interactions intraspécifiques ($\alpha_{11}$ et $\alpha_{22}$), c'est-à-dire l'effet de chaque espèce sur elle-même, vaut 1.<br>
Le nombre d'individus qui survivent à chaque génération est calculé de façon déterministe, c'est-à-dire à l'aide d'un taux fixe. Le taux de mortalité de l'espèce $i$ est $\mu_i$, donc le nombre d'individus de l'espèce $i$ qui survivent sur le site $x$ au pas de temps $t$ est : 
$$(N_{ix}(t)+Nb_{ix}(t))(1-\mu_i)$$
Le nombre d'individus qui dispersent parmis les survivant $D_{ix}(t)$ est aussi calculé de façon déterministe, avec $d_{i}$ le taux de dispesion de l'espèce $i$.

<div class="alert alert-block alert-info">
<b>Box 5 : Modèle de compétition de Lotka-Volterra</b> <br>
<p style='text-align: justify;'>Considérons 2 espèces et un site avec une capacitance K = 100 pour chacune des espèces (<b>Figure 8</b>). Lorsqu'une espèce est seule (ou que le coefficient d'interaction avec l'autre espèce vaut 0), elle va avoir une croissance logistique au sein du site et ses effectifs vont atteindre la capacitance du site (<b>Figure 8 A.</b>). Lorsque les 2 espèces sont présentes et en compétition, 2 cas simples se présentent. Si les coefficients d'interaction entre les espèces sont inférieurs ou égaux à 1 (l'effet de l'espèce i sur l'espèce j est inférieur à l'effet de l'espèce j sur elle-même), les 2 espèces vont coexister, mais leurs effectifs ne pourront atteindre qu'une fraction plus ou moins grande de la capacitance en fonction de la force de la compétition (<b>Figure 8 B.</b> et <b>C.</b>). Si l'un des coefficients est supérieur à 1 (l'effet de l'espèce i sur l'espèce j est supérieur à l'effet de l'espèce j sur elle-même), alors l'espèce pour laquelle ce coefficient est favorable va exclure l'autre espèce du site (exclusion compétitive) (<b>Figure 8 D.</b>). Nous verrons le cas où les coefficients d'interaction interspécifiques sont tous les deux supérieurs à 1 plus tard.
</p>

<figure>
<center><img src="LK.png" style="width:80%" /></center>
<figcaption align = "center"> <b>Figure 8.</b> Dynamiques des effectifs de 2 espèces selon un modèle de compétition de Lotka-Volterra pour une capacitance K = 100 et un nombre d'individus initial N0 = 0. <b>A.</b> Dynamique attendu lorsqu'une espèce est seule ou qu'il n'y a pas d'interaction entre les espèces. L'espèce 1 atteint la capacitance du milieu.<b>B.</b> Dynamique attendue lorsque les interaction interspécifiques sont équivalentes aux interactions intraspécifiques. Chaque individu de l'espèce 1 équivaut à un individu de l'espèce 2 lors du calcul du nombre d'individu de l'espèce 2 à la génération suivante, et réciproquement. Dans le cas où les 2 espèces débutent avec les mêmes effectifs, elles atteignent la moitié de la capcitance du milieu, la pression générée par l'autre espèce "occupant" l'autre moitiée.<b>C.</b> Dynamique attendue lorsque les interaction interspécifiques sont inférieures aux interactions intraspécifiques. Chaque individu de l'espèce 1 équivaut à 0.45 individu de l'espèce 2 lors du calcul du nombre d'individu de l'espèce 2 à la génération suivante, et chaque individu de l'espèce 2 équivaut à 0.55 individu de l'espèce 1 lors du calcul du nombre d'individu de l'espèce 1 à la génération suivante. Les effectifs chaque espèce s'équilibrent à une plus grande proportion de la capacitance que dans le cas précédent, puisque la pression exercée par l'autre espèce est plus faible.<b>D.</b> Dynamique attendue lorsqu'une des espèces domine l'autre. Chaque individu de l'espèce 2 équivaut à 2 individus de l'espèce 1 lors du calcul du nombre d'individu de l'espèce 1 à la génération suivante, donc l'espèce 2 exclut l'espèce 1 après quelques générations.
</figcaption>
</figure>
</div>

<p style='text-align: justify;'>La dynamique intra-site nous permet d'obtenir à chaque pas de temps un nombre de migrants pour chaque espèce pour chaque site allumé. Ces migrants sont ensuite soumis à une dynamique inter-site.
</p>

### 3. *Dynamique inter-site*

Définir nos 100 sites comme équidistants équivaut à dire que notre modèle est implicite dans l'espace. Pendant le processus de migration, la distance entre les sites n'a pas d'importance, et un individu issu de n'importe quel peut migrer vers n'importe quel site. Dans notre modèle, nous traduisons cela en fusionnant les migrants $D_{ix}(t)$ de espèce $i$ issus de chaque site $x$ en un pool global de migrants $D_i(t)$ (<b>Figure 9</b>). Nous appliquons un taux de mortalité $c_i$ à ce pool, qui correspond à la mortalité au cours de la migration pour l'espèce $i$. $c_i$ et $d_i$ constituent nos éléments de contrôle sur la limitation de la dispersion. Cette partie du sujet n'a pas encore été approfondie. Les individus du pool $D_i(t)$ sont ensuite répartis aléatoirement entre l'ensemble des 100 sites à l'aide d'une distribution multinomiale où chaque site a une probabilité de 1/100 d'être sélectionné. Pour chaque individu du pool, un site de destination parmi les 100 possibles est tiré au sort, et tous les sites ont la même probabilité d'être tirés. La somme des individus qui arrivent dans le site $x$ au temps $t$ est notée $I_{ix}(t)$.

<figure>
<center><img src="Dynamique.png" style="width:90%" /></center>
<figcaption align = "center"> <b>Figure 9.</b> Représentation des flux d'individus d'une espèce dans notre modèle pour un système à 5 sites sur 5 générations.</figcaption>
</figure>

### 4. *Génération d'une méta-population avec effets de priorité*

<p style='text-align: justify;'>Le dernier point de cette première étape est de déterminer quelle gamme de paramètres permet l'existence d'effets de priorité. Dans un modèle de Lotka-Volterra à 2 espèces équivalentes (r et K identiques), les effets de priorité sont possibles lorsque les coefficients d'interaction interspécifiques sont supérieurs aux coefficients d'interaction intraspécifiques (<b>Box 6</b>). Dans notre cas, nos 2 espèces ont des K différents selon les sites. Les sites de la ressource 1 ne nous intéressent pas ici, puisque l'espèce 2 ne peut pas y survivre. Nous nous concentrons sur les sites de la ressource 2, où l'espèce 2 a K = 200 et l'espèce 1 a K = g*200.
</p>

<div class="alert alert-block alert-info">
<b>Box 6 : Effet de priorité et coexistence dans un sytème à deux espèces</b> <br>
<p style='text-align: justify;'>
</p>

</div>

$$\rho=\sqrt{\frac{\alpha_{12}\alpha_{21}}{\alpha_{11}\alpha_{22}}} = \sqrt{\alpha_{12}\alpha_{21}}$$
$$\frac{f_1}{f_2}= \sqrt{\frac{a_{11}a_{12}}{a_{22}a_{21}}}= \sqrt{\frac{\frac{\alpha_{11}}{K_1}\frac{\alpha_{12}}{K_1}}{\frac{\alpha_{22}}{K_2}\frac{\alpha_{21}}{K_2}}}=\frac{K_2}{K_1}\sqrt{\frac{\alpha_{11}\alpha_{12}}{\alpha_{22}\alpha_{21}}} = \frac{K_0}{gK_0}\sqrt{\frac{\alpha_{12}}{\alpha_{21}}}=\frac{1}{g}\sqrt{\frac{\alpha_{12}}{\alpha_{21}}}$$

## **II. Les modèles de distribution conjointe des espèces (JSDM)**

In [ ]:
# jupyter nbconvert Presentation.ipynb --no-input --to html